In [1]:
#固有表現抽出は、テキストに出現する人名や地名などの固有名詞や、日付や時間などの数値表現を抽出する技術です。

In [2]:
#必要なPythonモジュール
#pip install numpy
#pip install scipy
#pip install sklearn
#pip install python-crfsuite

In [3]:
from itertools import chain
import pycrfsuite
import sklearn
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer

In [4]:
import codecs

class CorpusReader(object):

    def __init__(self, path):
        with codecs.open(path, encoding='utf-8') as f:
            sent = []
            sents = []
            for line in f:
                if line == '\n':
                    sents.append(sent)
                    sent = []
                    continue
                morph_info = line.strip().split('\t')
                sent.append(morph_info)
        train_num = int(len(sents) * 0.9)
        self.__train_sents = sents[:train_num]
        self.__test_sents = sents[train_num:]

    def iob_sents(self, name):
        if name == 'train':
            return self.__train_sents
        elif name == 'test':
            return self.__test_sents
        else:
            return None

In [5]:
c = CorpusReader('hironsan.txt')
train_sents = c.iob_sents('train')
test_sents = c.iob_sents('test')

In [6]:
train_sents[0]

[['2005', '名詞', '数', '*', '*', '*', '*', '*', 'B-DAT'],
 ['年', '名詞', '接尾', '助数詞', '*', '*', '*', '年', 'ネン', 'ネン', 'I-DAT'],
 ['7', '名詞', '数', '*', '*', '*', '*', '*', 'I-DAT'],
 ['月', '名詞', '一般', '*', '*', '*', '*', '月', 'ツキ', 'ツキ', 'I-DAT'],
 ['14', '名詞', '数', '*', '*', '*', '*', '*', 'I-DAT'],
 ['日', '名詞', '接尾', '助数詞', '*', '*', '*', '日', 'ニチ', 'ニチ', 'I-DAT'],
 ['、', '記号', '読点', '*', '*', '*', '*', '、', '、', '、', 'O'],
 ['南アフリカ',
  '名詞',
  '固有名詞',
  '地域',
  '国',
  '*',
  '*',
  '南アフリカ',
  'ミナミアフリカ',
  'ミナミアフリカ',
  'B-LOC'],
 ['共和', '名詞', '一般', '*', '*', '*', '*', '共和', 'キョウワ', 'キョーワ', 'I-LOC'],
 ['国', '名詞', '接尾', '一般', '*', '*', '*', '国', 'コク', 'コク', 'I-LOC'],
 ['ダーバン', '名詞', '固有名詞', '地域', '一般', '*', '*', 'ダーバン', 'ダーバン', 'ダーバン', 'I-LOC'],
 ['で', '助詞', '格助詞', '一般', '*', '*', '*', 'で', 'デ', 'デ', 'O'],
 ['開催', '名詞', 'サ変接続', '*', '*', '*', '*', '開催', 'カイサイ', 'カイサイ', 'O'],
 ['中', '名詞', '接尾', '副詞可能', '*', '*', '*', '中', 'チュウ', 'チュー', 'O'],
 ['の', '助詞', '連体化', '*', '*', '*', '*', 'の', 'ノ', 

In [7]:
#文字種の判定
#文字列に含まれるすべての文字種を-(ハイフン)で結合しています。


In [8]:
def is_hiragana(ch):
    return 0x3040 <= ord(ch) <= 0x309F

def is_katakana(ch):
    return 0x30A0 <= ord(ch) <= 0x30FF

def get_character_type(ch):
    if ch.isspace():
        return 'ZSPACE'
    elif ch.isdigit():
        return 'ZDIGIT'
    elif ch.islower():
        return 'ZLLET'
    elif ch.isupper():
        return 'ZULET'
    elif is_hiragana(ch):
        return 'HIRAG'
    elif is_katakana(ch):
        return 'KATAK'
    else:
        return 'OTHER'

def get_character_types(string):
    character_types = map(get_character_type, string)
    character_types_str = '-'.join(sorted(set(character_types)))

    return character_types_str

In [9]:
#品詞細分類の抽出
#形態素情報から品詞細分類を抽出する


In [10]:
def extract_pos_with_subtype(morph):
    idx = morph.index('*')

    return '-'.join(morph[1:idx])

In [11]:
#素性抽出
#各単語に対して素性抽出をする

In [12]:
def word2features(sent, i):
    word = sent[i][0]
    chtype = get_character_types(sent[i][0])
    postag = extract_pos_with_subtype(sent[i])
    features = [
        'bias',
        'word=' + word,
        'type=' + chtype,
        'postag=' + postag,
    ]
    if i >= 2:
        word2 = sent[i-2][0]
        chtype2 = get_character_types(sent[i-2][0])
        postag2 = extract_pos_with_subtype(sent[i-2])
        iobtag2 = sent[i-2][-1]
        features.extend([
            '-2:word=' + word2,
            '-2:type=' + chtype2,
            '-2:postag=' + postag2,
            '-2:iobtag=' + iobtag2,
        ])
    else:
        features.append('BOS')

    if i >= 1:
        word1 = sent[i-1][0]
        chtype1 = get_character_types(sent[i-1][0])
        postag1 = extract_pos_with_subtype(sent[i-1])
        iobtag1 = sent[i-1][-1]
        features.extend([
            '-1:word=' + word1,
            '-1:type=' + chtype1,
            '-1:postag=' + postag1,
            '-1:iobtag=' + iobtag1,
        ])
    else:
        features.append('BOS')

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        chtype1 = get_character_types(sent[i+1][0])
        postag1 = extract_pos_with_subtype(sent[i+1])
        features.extend([
            '+1:word=' + word1,
            '+1:type=' + chtype1,
            '+1:postag=' + postag1,
        ])
    else:
        features.append('EOS')

    if i < len(sent)-2:
        word2 = sent[i+2][0]
        chtype2 = get_character_types(sent[i+2][0])
        postag2 = extract_pos_with_subtype(sent[i+2])
        features.extend([
            '+2:word=' + word2,
            '+2:type=' + chtype2,
            '+2:postag=' + postag2,
        ])
    else:
        features.append('EOS')

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    return [morph[-1] for morph in sent]


def sent2tokens(sent):
    return [morph[0] for morph in sent]

In [1]:
#sent2featuresで文から素性を抽出します。実際に抽出される素性は以下のようになります。


In [14]:
>>> sent2features(train_sents[0])[0]
['bias',
 'word=2005',
 'type=ZDIGIT',
 'postag=名詞-数',
 'BOS',
 'BOS',
 '+1:word=年',
 '+1:type=OTHER',
 '+1:postag=名詞-接尾-助数詞',
 '+2:word=7',
 '+2:type=ZDIGIT',
 '+2:postag=名詞-数']

['bias',
 'word=2005',
 'type=ZDIGIT',
 'postag=名詞-数',
 'BOS',
 'BOS',
 '+1:word=年',
 '+1:type=OTHER',
 '+1:postag=名詞-接尾-助数詞',
 '+2:word=7',
 '+2:type=ZDIGIT',
 '+2:postag=名詞-数']

In [15]:
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

In [2]:
#モデルを学習するために、pycrfsuite.Trainerオブジェクトを作成し、学習データを読み込ませた後、trainメソッドを呼び出します

In [3]:
trainer = pycrfsuite.Trainer(verbose=False)

for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

NameError: name 'pycrfsuite' is not defined

In [18]:
#次に学習パラメータを設定します。

In [19]:
trainer.set_params({
    'c1': 1.0,   # coefficient for L1 penalty
    'c2': 1e-3,  # coefficient for L2 penalty
    'max_iterations': 50,  # stop earlier

    # include transitions that are possible, but not observed
    'feature.possible_transitions': True
})

In [20]:
trainer.train('model.crfsuite')

In [21]:
#学習したモデルを使用するためには、pycrfsuite.Taggerオブジェクトを作成し、学習したモデルを読み込み、tagメソッドを使用します。
#まずは、Taggerオブジェクトの作成と学習済みモデルの読み込みを行います。

In [22]:
tagger = pycrfsuite.Tagger()
tagger.open('model.crfsuite')

In [23]:
example_sent = test_sents[0]
print(' '.join(sent2tokens(example_sent)))

print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

昨年 10 月 に は 、 34 人 が 、 今回 の 現場 に 近い エジプト の タバ で 爆発 事件 の ため 死亡 し て いる 。
Predicted: B-DAT I-DAT I-DAT O O O O O O O O O O O O B-LOC O B-LOC O O O O O O O O O O
Correct:   B-DAT I-DAT I-DAT O O O O O O O O O O O O B-LOC O B-LOC O O O O O O O O O O


In [24]:
#モデルの評価
#作成したモデルについて評価していきましょう。評価は、適合率、再現率、F値で行います。


In [25]:
def bio_classification_report(y_true, y_pred):
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))

    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}

    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

In [26]:
#評価に使うためのテストデータ集合内の文に対してタグ付けします。

In [27]:
y_pred = [tagger.tag(xseq) for xseq in X_test]

In [28]:
#学習したモデルを用いてタグ付けしたデータと、正解データを評価用の関数に渡して結果を表示します。
#各カテゴリについて、適合率、再現率、F値、タグ数を表示しています。

In [29]:
>>> print(bio_classification_report(y_test, y_pred))
            

              precision    recall  f1-score   support

       B-ART       1.00      0.89      0.94         9
       I-ART       0.92      1.00      0.96        12
       B-DAT       1.00      1.00      1.00        12
       I-DAT       1.00      1.00      1.00        22
       B-LOC       1.00      0.95      0.97        55
       I-LOC       0.94      0.94      0.94        17
       B-ORG       0.75      0.86      0.80        14
       I-ORG       1.00      0.90      0.95        10
       B-PSN       0.00      0.00      0.00         3
       B-TIM       1.00      0.71      0.83         7
       I-TIM       1.00      0.81      0.90        16

   micro avg       0.96      0.91      0.94       177
   macro avg       0.87      0.82      0.84       177
weighted avg       0.95      0.91      0.93       177
 samples avg       0.14      0.14      0.14       177



D:\Anaconda\envs\tf2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\envs\tf2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\Anaconda\envs\tf2\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
